In [1]:
# importing libraries
import pandas as pd
import matplotlib as plt

In [6]:
# reading the csv file
data = pd.read_csv('Data/rain_data.csv')

In [8]:
print(data)

    EstacaoCodigo;NivelConsistencia;Data;TipoMedicaoChuvas;Maxima;Total;DiaMaxima;NumDiasDeChuva;MaximaStatus;TotalStatus;NumDiasDeChuvaStatus;TotalAnual;TotalAnualStatus;Chuva01;Chuva02;Chuva03;Chuva04;Chuva05;Chuva06;Chuva07;Chuva08;Chuva09;Chuva10;Chuva11;Chuva12;Chuva13;Chuva14;Chuva15;Chuva16;Chuva17;Chuva18;Chuva19;Chuva20;Chuva21;Chuva22;Chuva23;Chuva24;Chuva25;Chuva26;Chuva27;Chuva28;Chuva29;Chuva30;Chuva31;Chuva01Status;Chuva02Status;Chuva03Status;Chuva04Status;Chuva05Status;Chuva06Status;Chuva07Status;Chuva08Status;Chuva09Status;Chuva10Status;Chuva11Status;Chuva12Status;Chuva13Status;Chuva14Status;Chuva15Status;Chuva16Status;Chuva17Status;Chuva18Status;Chuva19Status;Chuva20Status;Chuva21Status;Chuva22Status;Chuva23Status;Chuva24Status;Chuva25Status;Chuva26Status;Chuva27Status;Chuva28Status;Chuva29Status;Chuva30Status;Chuva31Status  \
0                            2247057;1;01/12/1980;1;39                                                                                          